Assignment 7 By-
Sourima Dey, 
Rahul choudary Movva,
Sreepradh Dingari

In [ ]:
import tensorflow as tf
import os
import sys
import numpy as np
from tensorflow.keras import layers, models, optimizers
!pip install -q keras
import keras
#import regex
import unicodedata
import re
import io
from sklearn.model_selection import train_test_split
import time


mounting data in drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pathlib
path_to_file= pathlib.Path('/content/drive/MyDrive/Colab Notebooks/idl_colab/hin-eng/hin.txt')
print(path_to_file)

/content/drive/MyDrive/Colab Notebooks/idl_colab/hin-eng/hin.txt


Text processing for adding white space before punctuation..
adding a start and an end token to the sentence so that the model know when to start and stop predicting.

In [ ]:
def preprocess_sentence(w):
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)
  w = w.rstrip().strip()
  w = '<start> ' + w + ' <end>'
  return w

Extracing pair of sentences/words for input and target.
Input --> **Hindi**
Target--> **Engish**

hindi-english word pair separated by tab(“\t”)

In [ ]:
def create_dataset(path, num_examples):
    
    
    lines = io.open(path, ).read().strip().split('\n')[:-1]
    print(lines)
     
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:-1]]  for l in lines[:num_examples]]
    
    return zip(*word_pairs)

In [ ]:
lines = io.open(path_to_file, encoding='UTF-   8').read().strip().split('\n')[:-1]
print(lines)

['Wow!\tवाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #6179147 (fastrizwaan)', 'Duck!\tझुको!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #6179041 (fastrizwaan)', 'Duck!\tबतख़!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #6179042 (fastrizwaan)', 'Help!\tबचाओ!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #459377 (minshirui)', 'Jump.\tउछलो.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179121 (fastrizwaan)', 'Jump.\tकूदो.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179122 (fastrizwaan)', 'Jump.\tछलांग.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179123 (fastrizwaan)', 'Hello!\tनमस्ते।\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #480306 (minshirui)', 'Hello!\tनमस्कार।\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #480307 (minshirui)', 'Cheers!\tवाह-वाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #487006 (human600) & #61

In [ ]:
en, hi = create_dataset(path_to_file, None)
print(hi[8],en[8])

<start> नमस्कार। <end> <start> Hello ! <end>


In [ ]:
def max_length(tensor):
    return max(len(t) for t in tensor)


with tokenization we convert each unique word with its unique tokens in space.

- Initializing the tokenizer and fitting it to text
- finally converting that to sequences of vectors
- since each sentence is of different length so we need to pad it to make sequences of same length.

In [ ]:
def tokenize(lang):
  
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')   #filters: a string where each element is a character that will be filtered from the texts. 
  
  lang_tokenizer.fit_on_texts(lang)
  
  tensor = lang_tokenizer.texts_to_sequences(lang)
  
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')
  return tensor, lang_tokenizer

load the dataset, tokenize it and return output as tensors.

In [ ]:
def load_dataset(path, num_examples=None):
  targ_lang, inp_lang = create_dataset(path, num_examples)
  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)
  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [ ]:
num_examples = 2000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

#Calculate max_length of the tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)#
print(max_length(input_tensor))

['Wow!\tवाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #6179147 (fastrizwaan)', 'Duck!\tझुको!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #6179041 (fastrizwaan)', 'Duck!\tबतख़!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #6179042 (fastrizwaan)', 'Help!\tबचाओ!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #459377 (minshirui)', 'Jump.\tउछलो.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179121 (fastrizwaan)', 'Jump.\tकूदो.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179122 (fastrizwaan)', 'Jump.\tछलांग.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179123 (fastrizwaan)', 'Hello!\tनमस्ते।\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #480306 (minshirui)', 'Hello!\tनमस्कार।\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #480307 (minshirui)', 'Cheers!\tवाह-वाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #487006 (human600) & #61

Dividing dataset into train and validation

In [ ]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

1600 1600 400 400


index to word mapping of input and target

In [ ]:
def convert(lang, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, lang.index_word[t]))

In [ ]:
print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])

Input Language; index to word mapping
1 ----> <start>
19 ----> यह
864 ----> रास्ता
18 ----> बहुत
1473 ----> पतला
3 ----> है।
2 ----> <end>


In [ ]:
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])

Target Language; index to word mapping
1 ----> <start>
19 ----> this
555 ----> road
9 ----> is
47 ----> very
1180 ----> narrow
3 ----> .
2 ----> <end>


In [ ]:
#length of input_tensor_train
BUFFER_SIZE = len(input_tensor_train)
#batch size
BATCH_SIZE = 64
#steps taken in each epochs
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
#total embedding dimensions of tensor
embedding_dim = 256
units = 1024
#vocabulary size of input and target
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

#tf.data.Dataset.from_tensor.slices
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1
print(vocab_inp_size,vocab_tar_size)

2153 1684


In [ ]:
input_tensor_train, target_tensor_train = next(iter(dataset))
input_tensor_train.shape, target_tensor_train.shape

(TensorShape([64, 15]), TensorShape([64, 13]))

encoder neural network reads and encodes a source sentence into a fixed-length vector called a context vector.

-Embedding the vocabulary to the dimension we want.
-Passing output dimension(units) to GRU with glorot_uniform initializer.

In [ ]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
      super(Encoder, self).__init__()
      self.batch_sz = batch_sz
      self.enc_units = enc_units
      self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
      self.gru = tf.keras.layers.GRU(self.enc_units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')
  def call(self, x, hidden):
      x = self.embedding(x)
      output, state = self.gru(x, initial_state = hidden)
      return output, state
  def initialize_hidden_state(self):
      return tf.zeros((self.batch_sz, self.enc_units))

In [ ]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)


at time t we consider about t-1 hidden state of the decoder. Then we calculate alignment , context vectors. And then we concatenate this context with hidden state of the decoder at t-1. So before the softmax this concatenated vector goes inside a GRU.

In [ ]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
      super(BahdanauAttention, self).__init__()
      self.W1 = tf.keras.layers.Dense(units)
      self.W2 = tf.keras.layers.Dense(units)
      self.V = tf.keras.layers.Dense(1)
    def call(self, query, values):
      hidden_with_time_axis = tf.expand_dims(query, 1)
      score = self.V(tf.nn.tanh(
              self.W1(values) + self.W2(hidden_with_time_axis)))
      attention_weights = tf.nn.softmax(score, axis=1)
      context_vector = attention_weights * values
      context_vector = tf.reduce_sum(context_vector, axis=1)
      return context_vector, attention_weights

concatenating previous state of decoder with attention context vector.
Passing output of previous state to GRU
Reshaping the output produced by gru and finally passing it to fully connected layer

In [ ]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)
    
    self.attention = BahdanauAttention(self.dec_units)
  def call(self, x, hidden, enc_output):
      
      context_vector, attention_weights = self.attention(hidden, enc_output)
      x = self.embedding(x)
      x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
      output, state = self.gru(x)

      output = tf.reshape(output, (-1, output.shape[2]))

      x = self.fc(output)
      return x, state, attention_weights

In [ ]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

the optimizer and the loss function

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

Train the model

In [ ]:
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.1701
Epoch 1 Loss 3.2675
Time taken for 1 epoch 109.72517085075378 sec

Epoch 2 Batch 0 Loss 2.8560
Epoch 2 Loss 2.6112
Time taken for 1 epoch 89.43593645095825 sec

Epoch 3 Batch 0 Loss 2.4502
Epoch 3 Loss 2.4339
Time taken for 1 epoch 89.03547143936157 sec

Epoch 4 Batch 0 Loss 2.4518
Epoch 4 Loss 2.2839
Time taken for 1 epoch 89.58126306533813 sec

Epoch 5 Batch 0 Loss 2.0360
Epoch 5 Loss 2.1612
Time taken for 1 epoch 89.26661682128906 sec

Epoch 6 Batch 0 Loss 2.2263
Epoch 6 Loss 2.0416
Time taken for 1 epoch 90.594153881073 sec

Epoch 7 Batch 0 Loss 1.8516
Epoch 7 Loss 1.9092
Time taken for 1 epoch 90.82380294799805 sec

Epoch 8 Batch 0 Loss 1.7638
Epoch 8 Loss 1.7907
Time taken for 1 epoch 90.5342755317688 sec

Epoch 9 Batch 0 Loss 1.5891
Epoch 9 Loss 1.6857
Time taken for 1 epoch 89.5019371509552 sec

Epoch 10 Batch 0 Loss 1.4817
Epoch 10 Loss 1.5852
Time taken for 1 epoch 89.54297971725464 sec



Evaluation of model

In [ ]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    sentence = preprocess_sentence(sentence)

    inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                           maxlen=max_length_inp,
                                                           padding='post')
    inputs = tf.convert_to_tensor(inputs)

    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()

        predicted_id = tf.argmax(predictions[0]).numpy()

        result += targ_lang.index_word[predicted_id] + ' '

        if targ_lang.index_word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence, attention_plot

In [ ]:
def translate(sentence):
    result, sentence, attention_plot = evaluate(sentence)

    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    #attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    #plot_attention(attention_plot, sentence.split(' '), result.split(' '))

In [ ]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [ ]:
translate(u'मैं ठीक हूँ।')

Input: <start> मैं ठीक हूँ। <end>
Predicted translation: i don't know . <end> 


- Which parts of the sentence are used as a token? Each character, each word, or are some words split up?
each word
- Do the same tokens in different language have the same ID?
No 
- What is the relation between the encoder output and the encoder hidden state which is used to initialize the decoder hidden state

Encoder hiddent state outputs the context vector which is fed into the decoder.

- Is the decoder attending to all previous positions, including the previous decoder predictions?
 
 Yes it takes the previous output to generate the next word

- Does the encoder output change in different decoding steps?

No 

- Does the context vector change in different decoding steps?

No 

- The decoder uses teacher forcing. Does this mean the time steps can be computed in parallel?

- Why is a mask applied to the loss function?

- When translating the same sentence multiple times, do you get the same result? Why (not)? If not, what changes need to be made to get the same result each time?